# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv exported in Part 1
part_1_csv = "../output_data/cleaned_city_data.csv"

part_1_df = pd.read_csv(part_1_csv)

weather_dataframe = part_1_df.drop(columns=["City_ID"])
weather_dataframe.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,butaritari,3.0707,172.7902,81.45,71.0,51.0,7.29,KI,1.666491e+09
1,shieli,44.1667,66.7500,43.93,74.0,31.0,5.70,KZ,1.666492e+09
2,new norfolk,-42.7826,147.0587,56.57,74.0,100.0,4.00,AU,1.666491e+09
3,chokurdakh,70.6333,147.9167,12.60,95.0,100.0,10.78,RU,1.666491e+09
4,alofi,-19.0595,-169.9187,77.79,79.0,100.0,10.09,NU,1.666491e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Set latitude and longitude as the locations
locations = weather_dataframe[["Lat", "Lng"]]

# Set humidity as the weight
humidity = weather_dataframe["Humidity"].astype(float)

# Add a humidity heatmap layer to the map
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=max(weather_dataframe["Humidity"]),
                                   point_radius=2)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find my ideal weather condition
filtered_weather_dataframe = weather_dataframe

# Drop any rows that don't contain all three conditions; want to be sure the weather is ideal

# A max temperature lower than 80 degrees but higher than 70
filtered_weather_dataframe = filtered_weather_dataframe.loc[(filtered_weather_dataframe["Max Temp"] < 80) & (filtered_weather_dataframe["Max Temp"] > 70)]

# Wind speed less than 10 mph
filtered_weather_dataframe = filtered_weather_dataframe.loc[filtered_weather_dataframe["Wind Speed"] < 10]

# Zero cloudiness
filtered_weather_dataframe = filtered_weather_dataframe.loc[filtered_weather_dataframe["Cloudiness"] == 0]

# Drop rows with null values
filtered_weather_dataframe = filtered_weather_dataframe.dropna()

filtered_weather_dataframe

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,sur,22.5667,59.5289,76.84,54.0,0.0,6.33,OM,1.666492e+09
155,fort smith,35.3859,-94.3986,75.25,56.0,0.0,5.01,US,1.666491e+09
185,carnarvon,-24.8667,113.6333,72.64,63.0,0.0,8.66,AU,1.666492e+09
230,sakakah,29.9697,40.2064,71.42,37.0,0.0,6.89,SA,1.666491e+09
318,nsanje,-16.9200,35.2620,72.77,82.0,0.0,4.03,MW,1.666492e+09
350,sancti spiritus,21.9297,-79.4425,71.96,91.0,0.0,5.32,CU,1.666492e+09
351,adrar,20.5022,-10.0711,79.88,17.0,0.0,7.02,MR,1.666492e+09
412,alexandria,31.2156,29.9553,70.92,70.0,0.0,6.29,EG,1.666492e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store dataframe into variable named hotel_df
hotel_df = filtered_weather_dataframe

# Setting paramameters dictionary to update after each iteration
parameters = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get latitude and longitude from dataframe
    latitude = row["Lat"]
    longitude = row["Lng"]

    # change location every iteration
    parameters["location"] = f"{latitude},{longitude}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=parameters).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"The closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 45: sur.
The closest hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 155: fort smith.
The closest hotel is Hampton Inn Fort Smith.
------------
Retrieving Results for Index 185: carnarvon.
The closest hotel is Hospitality Carnarvon.
------------
Retrieving Results for Index 230: sakakah.
The closest hotel is Fakhamat Al Orjoana Apartment.
------------
Retrieving Results for Index 318: nsanje.
The closest hotel is Nsanje Discovery Lodge.
------------
Retrieving Results for Index 350: sancti spiritus.
The closest hotel is Hostal El Parque.
------------
Retrieving Results for Index 351: adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 412: alexandria.
The closest hotel is فندق قاعود.
------------


In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,sur,22.5667,59.5289,76.84,54.0,0.0,6.33,OM,1.666492e+09,Sur Plaza Hotel
155,fort smith,35.3859,-94.3986,75.25,56.0,0.0,5.01,US,1.666491e+09,Hampton Inn Fort Smith
185,carnarvon,-24.8667,113.6333,72.64,63.0,0.0,8.66,AU,1.666492e+09,Hospitality Carnarvon
230,sakakah,29.9697,40.2064,71.42,37.0,0.0,6.89,SA,1.666491e+09,Fakhamat Al Orjoana Apartment
318,nsanje,-16.9200,35.2620,72.77,82.0,0.0,4.03,MW,1.666492e+09,Nsanje Discovery Lodge
350,sancti spiritus,21.9297,-79.4425,71.96,91.0,0.0,5.32,CU,1.666492e+09,Hostal El Parque
351,adrar,20.5022,-10.0711,79.88,17.0,0.0,7.02,MR,1.666492e+09,NaN
412,alexandria,31.2156,29.9553,70.92,70.0,0.0,6.29,EG,1.666492e+09,فندق قاعود


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Create the marker layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create the hotel symbol layer
hotel_symbol_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Add marker layer ontop of heat map
fig.add_layer(markers)
fig.add_layer(hotel_symbol_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))